# Kepler's Laws of Planetary Motion

This notebook contains the programmatic verification for the **Kepler's Laws of Planetary Motion** entry from the THEORIA dataset.

**Entry ID:** keplers_laws  
**Required Library:** sympy 1.12.0

## Description
Kepler's three laws describe planetary motion around the Sun: orbits are ellipses with the Sun at one focus, planets sweep equal areas in equal times, and the square of orbital period is proportional to the cube of the semi-major axis. These empirical laws provided the foundation for Newton's law of gravitation and remain fundamental to celestial mechanics and orbital dynamics.

## Installation
First, let's install the required library:

In [ ]:
# Install required library with exact version
!pip install sympy==1.12.0

## Programmatic Verification

The following code verifies the derivation mathematically:

In [ ]:
import sympy as sp

# ===============================
# Programmatic verification (step-by-step) for Kepler's Laws
# Python 3.11, SymPy 1.12
# Each block mirrors the derivation steps and includes asserts.
# ===============================

# Common symbols
t = sp.symbols('t', real=True)
G, M, m = sp.symbols('G M m', positive=True, real=True)
a, e, h = sp.symbols('a e h', positive=True, real=True)
theta = sp.symbols('theta', real=True)
pi = sp.pi

# --- Step 1-2: Central (radial) force and zero torque ---------------------
# Represent a radial direction by an arbitrary vector u (unit length not required for the cross-product identity).
u1, u2, u3 = sp.symbols('u1 u2 u3', real=True)
u = sp.Matrix([u1, u2, u3])
r = sp.symbols('r', positive=True, real=True)
r_vec = r * u
F_vec = -G*M*m/r**2 * u   # gravitational force is parallel to r
tau_vec = sp.simplify(r_vec.cross(F_vec))
assert tau_vec == sp.Matrix([0,0,0])  # torque is zero for parallel vectors
print('OK: Step 1-2 – torque τ = r×F vanishes for a central force.')

# --- Step 3-6: Angular momentum conservation and areal velocity -----------
# From τ = 0 ⇒ dL/dt = 0. We also verify dA/dt = (1/2)|r×v| = L/(2m).
r_t = sp.Function('r')(t)
th_t = sp.Function('theta')(t)
x = r_t*sp.cos(th_t)
y = r_t*sp.sin(th_t)
r_cart = sp.Matrix([x, y, 0])
v_cart = sp.diff(r_cart, t)
cross_rv_z = sp.simplify(sp.Matrix([0,0,1]).dot(r_cart.cross(v_cart)))  # z-component of r×v
assert sp.simplify(cross_rv_z - r_t**2*sp.diff(th_t, t)) == 0
# Angular momentum (planar): L_z = m * (r×v)_z  ⇒  dA/dt = (1/2)*(r×v)_z = L/(2m)
print('OK: Step 5-6 – areal velocity dA/dt = L/(2m) is constant.')

# --- Step 7-18: Binet equation via θ as independent variable --------------
# Use h := r^2*θ̇ (specific angular momentum, constant). With u(θ) := 1/r,
# we get ṙ = -h u', r̈ = -h^2 u^2 u'' and r θ̇^2 = h^2 u^3.
u = sp.Function('u')(theta)
ud = sp.diff(u, theta)
udd = sp.diff(u, theta, 2)
theta_dot = h*u**2
drdtheta = sp.diff(1/u, theta)           # = -u'/u^2
drdt = sp.simplify(drdtheta * theta_dot)  # = -h*u'
d2rdt2 = sp.simplify(sp.diff(drdtheta * theta_dot, theta) * theta_dot)  # chain rule ⇒ -h^2 u^2 u''
centrifugal = sp.simplify((1/u) * theta_dot**2)  # r*θ̇^2 = h^2 u^3
# Radial EOM: r̈ - r θ̇^2 = -GM/r^2  ⇒  (-h^2 u^2 u'') - h^2 u^3 = -GM u^2
expr = sp.simplify((d2rdt2 - centrifugal + G*M*u**2)/(-h**2*u**2))
assert sp.simplify(expr - (udd + u - G*M/h**2)) == 0
print('OK: Step 18 – Binet equation u" + u = GM/h² obtained.')

# --- Step 19-25: Solve orbit shape and express r(θ) -----------------------
C1, C2 = sp.symbols('C1 C2', real=True)
ode = sp.Eq(sp.diff(sp.Function('u')(theta), (theta, 2)) + sp.Function('u')(theta), G*M/h**2)
sol = sp.dsolve(ode)  # u(θ) = C1 sinθ + C2 cosθ + GM/h²
# Choose periapsis at θ=0 ⇒ C1=0 and define e so that C2 = e*(GM/h²).
u_sol = (G*M/h**2)*(1 + e*sp.cos(theta))
p = sp.simplify(h**2/(G*M))
r_theta = sp.simplify(1/u_sol)
r_conic = sp.simplify(p/(1 + e*sp.cos(theta)))
assert sp.simplify(r_theta - r_conic) == 0
print('OK: Step 20-22 – conic solution r(θ) = p / (1 + e cosθ).')

# --- Step 23-26: Ellipse relations, semi-major axis, and h² relation ------
r_p = sp.simplify(r_conic.subs(theta, 0))
r_a = sp.simplify(r_conic.subs(theta, sp.pi))
assert sp.simplify(r_p - p/(1+e)) == 0 and sp.simplify(r_a - p/(1-e)) == 0
a_from_ra_rp = sp.simplify((r_p + r_a)/2)         # ⇒ p/(1-e²)
assert sp.simplify(a_from_ra_rp - p/(1 - e**2)) == 0
assert sp.simplify(h**2 - G*M*(p)) == 0           # by definition of p
assert sp.simplify(h**2 - G*M*a_from_ra_rp*(1 - e**2)) == 0  # substituting a = p/(1-e²)
print('OK: Step 23-26 – ellipse geometry and h² = GM a (1-e²).')

# --- Step 27-32: Area, period, and Kepler's 3rd law -----------------------
b = a*sp.sqrt(1 - e**2)
A_ellipse = sp.simplify(pi*a*b)                    # πab
T = sp.simplify(A_ellipse / (h/2))                 # using dA/dt = h/2
T_expected = sp.simplify(2*pi*a**(sp.Rational(3,2)) / sp.sqrt(G*M))
assert sp.simplify(T.subs(h, sp.sqrt(G*M*a*(1-e**2))) - T_expected) == 0
assert sp.simplify(T_expected**2 - (4*pi**2*a**3)/(G*M)) == 0
print('OK: Step 27-32 – T = 2π a^{3/2}/√(GM) and T² = 4π² a³/(GM).')

# --- Additional consistency checks ---------------------------------------
a_var = sp.symbols('a_var', positive=True)
r_kepler1 = sp.simplify((a_var*(1 - e**2))/(1 + e*sp.cos(theta)))
assert sp.simplify(r_kepler1.subs(theta, 0) - a_var*(1-e)) == 0
assert sp.simplify(r_kepler1.subs(theta, pi) - a_var*(1+e)) == 0
assert sp.simplify(r_kepler1.subs(e, 0) - a_var) == 0
print('OK: Special cases – periapsis, apoapsis, and circular limit e=0.')

# Solar-system sanity check: Earth vs Mars (units: AU and years)
a_earth, a_mars = sp.Rational(1,1), sp.Rational(152,100)
T_earth, T_mars = sp.Rational(1,1), sp.Rational(188,100)
ratio_periods  = sp.nsimplify((T_earth**2) / (T_mars**2))
ratio_distances = sp.nsimplify((a_earth**3) / (a_mars**3))
assert abs(float(ratio_periods - ratio_distances)) < 0.01
print('OK: Numerical check – (T₁/T₂)² ≈ (a₁/a₂)³ for Earth/Mars (≤1% error).')

print('All Kepler verifications passed ✔')


## Source

📖 **View this entry:** [theoria-dataset.org/entries.html?entry=keplers_laws.json](https://theoria-dataset.org/entries.html?entry=keplers_laws.json)

This verification code is part of the [THEORIA dataset](https://github.com/theoria-dataset/theoria-dataset), a curated collection of theoretical physics derivations with programmatic verification.

**License:** CC-BY 4.0